# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import joblib
import os

from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier
from utils import CustomCleaner

import sklearn; sklearn.set_config(display='diagram')

import os
import sys
sys.stderr = open(os.devnull, "w")  # silence stderr
# sys.stderr = sys.__stderr__

ImportError: cannot import name 'load_data' from 'utils' (/Users/carlos.costa/studies/MDSAA-NOVA-IMS/machine_learning/project/project_vitor/notebooks/utils.py)

# Load data

In [ ]:
df = pd.read_csv('../raw_data/train.csv')
df.shape

# Modelos

In [ ]:
preproc = joblib.load('../pipelines/preproc_pipeline_v1')
preproc

* Logistic Reg
* SVC
* KNN
* DT
* RF
* NB
* GBC
* Adaboost

## Log

In [ ]:
X_train, X_val, y_train, y_val = load_data(df, undersampling=True)

log_pipe = Pipeline([
    ('preproc', preproc),
    ('log', LogisticRegression(n_jobs=-1))
])

cross_val_score(log_pipe, X_train, y_train, cv=10, n_jobs=-1, scoring='f1', verbose=1)

## KNN

In [ ]:
X_train, X_val, y_train, y_val = load_data(df, undersampling=True)

knn_pipe = Pipeline([
    ('preproc', preproc),
    ('knn', KNeighborsClassifier(n_jobs=-1))
])

cross_val_score(knn_pipe, X_train, y_train, cv=10, n_jobs=-1, scoring='f1', verbose=1)

## SVC

In [ ]:
X_train, X_val, y_train, y_val = load_data(df, undersampling=True)

svc_pipe = Pipeline([
    ('preproc', preproc),
    ('svc', SVC(C=2))
])

# cross_val_score(knn_pipe, X_train, y_train, cv=10, n_jobs=-1, scoring='f1').mean()
svc_pipe.fit(X_train, y_train)
y_pred = svc_pipe.predict(X_val)
f1_score(y_val, y_pred)

## DT

In [ ]:
X_train, X_val, y_train, y_val = load_data(df, undersampling=True)

dt_pipe = Pipeline([
    ('preproc', preproc),
    ('dt', DecisionTreeClassifier())
])

cross_val_score(dt_pipe, X_train, y_train, cv=10, n_jobs=-1, scoring='f1')

## Ada

In [ ]:
X_train, X_val, y_train, y_val = load_data(df, undersampling=True)

ada_pipe = Pipeline([
    ('preproc', preproc),
    ('ada', AdaBoostClassifier(n_estimators=100, learning_rate=1))
])

cross_val_score(ada_pipe, X_train, y_train, cv=10, n_jobs=-1, scoring='f1')

## RF

In [ ]:
X_train, X_val, y_train, y_val = load_data(df, undersampling=True)

rf_pipe = Pipeline([
    ('preproc', preproc),
    ('ada', RandomForestClassifier(n_estimators=300, n_jobs=-1))
])

cross_val_score(rf_pipe, X_train, y_train, cv=10, n_jobs=-1, scoring='f1')

## GaussianNB

In [ ]:
X_train, X_val, y_train, y_val = load_data(df, undersampling=True)

nb_pipe = Pipeline([
    ('preproc', preproc),
    ('nb', GaussianNB())
])

cross_val_score(nb_pipe, X_train, y_train, cv=10, n_jobs=-1, scoring='f1')

## GradientBoostC

In [ ]:
X_train, X_val, y_train, y_val = load_data(df, undersampling=True)

gbc_pipe = Pipeline([
    ('preproc', preproc),
    ('gbc', GradientBoostingClassifier())
])

cross_val_score(gbc_pipe, X_train, y_train, cv=10, n_jobs=-1, scoring='f1')

## Stochastic Gradient Descent

In [ ]:
X_train, X_val, y_train, y_val = load_data(df, undersampling=True)

sgd_pipe = Pipeline([
    ('preproc', preproc),
    ('sgd', GradientBoostingClassifier())
])

cross_val_score(sgd_pipe, X_train, y_train, cv=10, n_jobs=-1, scoring='f1')

# Submit

### Baseline

In [ ]:
X_test = pd.read_csv('../raw_data/test.csv')

df_sub = pd.DataFrame({'readmitted_binary': np.zeros(X_test.shape[0])}, index=indices)
df_sub.readmitted_binary = df_sub.readmitted_binary.replace({0: 'No', 1: 'Yes'})
df_sub

### Models

In [ ]:
X_train, X_val, y_train, y_val = load_data(df, undersampling=True)
X_test = pd.read_csv('../raw_data/test.csv')

indices = X_test.encounter_id
# svc_pipe.fit(X_train, y_train)
y_sub = svc_pipe.predict(X_test)
df_sub = pd.DataFrame({'readmitted_binary': y_sub}, index=indices)
df_sub.readmitted_binary = df_sub.readmitted_binary.replace({0: 'No', 1: 'Yes'})

In [ ]:
df_sub.readmitted_binary.value_counts()

In [ ]:
df_sub.to_csv('../submissions/.csv')